# Klavis MCP Sandbox API Example

This notebook demonstrates the core sandbox lifecycle endpoints:
- **Acquire** a sandbox instance
- **Get** sandbox details  
- **Initialize** with test data
- **Reset** to initial state
- **Release** the sandbox

## 1. Setup

In [ ]:
import os
import httpx
from google.colab import userdata

BASE_URL = "https://api.klavis.ai"
KLAVIS_API_KEY = userdata.get("KLAVIS_API_KEY")

headers = {"Authorization": f"Bearer {KLAVIS_API_KEY}"}

# TODO: Choose your {server_name}
SERVER_NAME = "google_docs"

## 2. Acquire Sandbox

`POST /sandbox/{server_name}` - Acquire an idle sandbox instance

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.post(
        f"{BASE_URL}/sandbox/{SERVER_NAME}",
        headers=headers
    )

SANDBOX_ID = resp.json()["sandbox_id"]
print(resp.json())

{'sandbox_id': 'bafbc2b7-0fe6-427a-ac15-e75b2c663588', 'server_urls': {'google_docs': 'https://gdocs-mcp-server.klavis.ai/mcp/?instance_id=940e462c-9afa-4a95-ac28-364f0b1e3afe'}, 'server_name': 'google_docs', 'status': 'occupied', 'message': 'Sandbox acquired successfully. Use server_urls to connect to the MCP server.'}


## 3. Get Sandbox Details

`GET /sandbox` - Get sandbox info that you occupied

`GET /sandbox/{server_name}/{sandbox_id}` - Get sandbox information

In [ ]:
# Get your sandbox
async with httpx.AsyncClient() as client:
    resp = await client.get(
        f"{BASE_URL}/sandbox",
        headers=headers
    )

print(resp.json())

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.get(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}",
        headers=headers
    )

print(resp.json())

{'sandbox_id': 'bafbc2b7-0fe6-427a-ac15-e75b2c663588', 'server_url': 'https://gdocs-mcp-server.klavis.ai/mcp/?instance_id=940e462c-9afa-4a95-ac28-364f0b1e3afe', 'server_name': 'google_docs', 'status': 'occupied', 'updated_at': '2026-01-29T12:09:44.719527-08:00'}


## 4. Initialize Sandbox with Data

`POST /sandbox/{server_name}/{sandbox_id}/initialize` - Initialize with data

In [ ]:
# Example data
google_docs_data = {
    "documents": [
        {
            "title": "Team Meeting Agenda",
            "content": "# Team Meeting Agenda\n\n## Date: January 15, 2025\n\n### Topics:\n1. Q1 Goals Review\n2. Project Updates\n3. Resource Allocation\n4. Action Items\n\n### Attendees:\n- John Doe (Project Manager)\n- Jane Smith (Tech Lead)\n- Bob Johnson (Designer)\n- Alice Williams (QA Lead)\n\n### Notes:\nDiscuss upcoming product launch and timeline adjustments."
        },
        {
            "title": "Product Roadmap 2025",
            "content": "# Product Roadmap 2025\n\n## Q1 (January - March)\n- Feature A development and testing\n- Beta program launch\n- User feedback collection\n\n## Q2 (April - June)\n- Feature B public launch\n- Mobile app development begins\n- Integration with third-party services\n\n## Q3 (July - September)\n- Performance optimization sprint\n- Mobile app beta release\n- Advanced analytics implementation\n\n## Q4 (October - December)\n- Year-end feature freeze\n- Comprehensive security audit\n- Planning for 2026 initiatives"
        },
        {
            "title": "Project Proposal - Customer Portal",
            "content": "# Project Proposal: Customer Portal\n\n## Executive Summary\nThis proposal outlines the development of a new customer self-service portal designed to improve user experience and reduce support ticket volume.\n\n## Business Objectives\n1. Enhance customer self-service capabilities\n2. Reduce support workload by 30%\n3. Improve customer satisfaction scores by 25%\n4. Enable 24/7 account management access\n\n## Technical Requirements\n- Responsive web design\n- Mobile-first approach\n- Integration with existing CRM\n- Real-time data synchronization\n\n## Project Timeline\n- Planning Phase: 2 weeks\n- Design Phase: 3 weeks\n- "
        }
    ]
}

async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.post(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}/initialize",
        headers=headers,
        json=google_docs_data
    )

print(resp.json())

{'sandbox_id': 'bafbc2b7-0fe6-427a-ac15-e75b2c663588', 'status': 'occupied', 'message': 'Initialized sandbox with 3 records', 'records_created': {'documents': 3}}


## 5. Export Sandbox

`GET /sandbox/{server_name}/{sandbox_id}/dump` -Export all data from the sandbox

In [ ]:
import json

async with httpx.AsyncClient() as client:
    resp = await client.get(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}/dump",
        headers=headers
    )

print(json.dumps(resp.json(), indent=2))

{
  "sandbox_id": "bafbc2b7-0fe6-427a-ac15-e75b2c663588",
  "server_name": "google_docs",
  "dumped_at": "2026-01-29T20:14:10.662994",
  "data": {
    "documents": [
      {
        "title": "Project Proposal - Customer Portal",
        "content": "# Project Proposal: Customer Portal\n\n## Executive Summary\nThis proposal outlines the development of a new customer self-service portal designed to improve user experience and reduce support ticket volume.\n\n## Business Objectives\n1. Enhance customer self-service capabilities\n2. Reduce support workload by 30%\n3. Improve customer satisfaction scores by 25%\n4. Enable 24/7 account management access\n\n## Technical Requirements\n- Responsive web design\n- Mobile-first approach\n- Integration with existing CRM\n- Real-time data synchronization\n\n## Project Timeline\n- Planning Phase: 2 weeks\n- Design Phase: 3 weeks\n- "
      },
      {
        "title": "Product Roadmap 2025",
        "content": "# Product Roadmap 2025\n\n## Q1 (January 

## 6. Reset Sandbox

`POST /sandbox/{server_name}/{sandbox_id}/reset` - Clear all data, for re-use this sandbox

In [ ]:
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.post(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}/reset",
        headers=headers
    )

print(resp.json())

{'sandbox_id': 'bafbc2b7-0fe6-427a-ac15-e75b2c663588', 'status': 'occupied', 'message': 'Reset completed. Deleted 3 documents.'}


## 7. Release Sandbox

`DELETE /sandbox/{server_name}/{sandbox_id}` - Release back to idle pool

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.delete(
        f"{BASE_URL}/sandbox/{SERVER_NAME}/{SANDBOX_ID}",
        headers=headers
    )

print(resp.json())

{'sandbox_id': 'bafbc2b7-0fe6-427a-ac15-e75b2c663588', 'status': 'idle', 'message': 'Sandbox bafbc2b7-0fe6-427a-ac15-e75b2c663588 released successfully and is now available for reuse'}
